In [136]:
import tensorflow as tf






In [128]:

tr_data = tf.keras.preprocessing.image_dataset_from_directory("AlzheimerDataset/train",image_size=(178,208),label_mode="categorical")
te_data = tf.keras.preprocessing.image_dataset_from_directory("AlzheimerDataset/test",image_size=(178,208),label_mode="categorical")



Found 5121 files belonging to 4 classes.
Found 1279 files belonging to 4 classes.


In [129]:

#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(178,208,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

In [102]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [106]:
#train the model
model.fit(datast,labelst, epochs=3)

Epoch 1/3
1/1 [==============================] - 0s 998us/step - loss: 18.0226 - accuracy: 0.1250
Epoch 2/3
1/1 [==============================] - 0s 2ms/step - loss: 787.1192 - accuracy: 0.5625
Epoch 3/3
1/1 [==============================] - 0s 2ms/step - loss: 1683.2361 - accuracy: 0.4062


In [130]:
train = tr_data.as_numpy_iterator()
test= te_data.as_numpy_iterator()
    

In [145]:
n = list(train)
b = list(test)

In [132]:
labelst = n[0][1]
datast = n[0][0]

tes

In [133]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten,Dropout


## CHANGE MODEL STRUCTURE ##
layers_unlocked = True
vg_model = tf.keras.applications.vgg16.VGG16(include_top=False,input_shape=(178,208,3), pooling = 'max') #added pooling
# retrain some of the convolutional layers deep in the model, but not the fully connected layers at the end
vg_model.get_layer('block1_conv1').trainable = layers_unlocked
vg_model.get_layer('block1_conv2').trainable = layers_unlocked
vg_model.get_layer('block2_conv1').trainable = layers_unlocked
vg_model.get_layer('block2_conv2').trainable = layers_unlocked
vg_model.get_layer('block3_conv1').trainable = layers_unlocked
vg_model.get_layer('block3_conv2').trainable = layers_unlocked
vg_model.get_layer('block3_conv3').trainable = layers_unlocked
vg_model.get_layer('block4_conv1').trainable = layers_unlocked
vg_model.get_layer('block4_conv2').trainable = layers_unlocked
vg_model.get_layer('block4_conv3').trainable = layers_unlocked




## Add new trainable FC layers ##
flat = Flatten()(vg_model.output)
fc1 = Dense(1024,activation='relu', kernel_initializer='he_uniform')(flat) # put in kernel initializer he-uniform
dp1 = Dropout(0.25)(fc1)                                                   # changed dropout here from .5
output = Dense(4,activation='softmax')(dp1)                                # changed to sigmoid from softmax
vg_model = Model(inputs=vg_model.inputs, outputs=output)

vg_model.summary()

vg_model.load_weights('val20_epochs50.h5')

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 178, 208, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 178, 208, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 178, 208, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 89, 104, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 89, 104, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 89, 104, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 44, 52, 128)      

In [134]:
vg_model.save('attempt.h5')

In [147]:
vg_model.compile("adam","categorical_crossentropy")
print(vg_model.evaluate(te_data))

11/40 [=======>......................] - ETA: 3:04 - loss: 24.1299

KeyboardInterrupt: 

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 